# Домашнее задание № 3. Исправление опечаток

## 1. Доп. ранжирование по вероятности (3 балла)

Дополните get_closest_hybrid_match в семинаре так, чтобы из кандадатов с одинаковым расстоянием редактирования выбиралось наиболее вероятное.

In [3]:
!pip install textdistance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import os, re
from string import punctuation
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity, cosine_distances
from sklearn.metrics import classification_report
from collections import Counter
import textdistance
from difflib import get_close_matches
import pandas as pd
from tqdm.notebook import tqdm

In [5]:
import logging
import os
import shutil

import numpy as np
from google.colab import drive

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)  # гарантируем воспроизводимость

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)
logger.info('Инициализировали логгер')

ROOT_DIR = '/content/drive'
drive.mount(ROOT_DIR)
logger.info('Подключили диск')

root_data_dir = os.path.join(ROOT_DIR, 'MyDrive', 'Colab Notebooks')
if not os.path.exists(root_data_dir):
  raise RuntimeError('Отсутствует директория с данными')
else:
  logger.info('Содержимое директории %s: %s', root_data_dir, os.listdir(root_data_dir))

Mounted at /content/drive


In [19]:

file = os.path.join(root_data_dir, 'wiki_data.txt')
corpus = open(file, encoding='utf8').read()
vocab = Counter(re.findall('[а-яА-ЯёЁ]+', corpus.lower()))

word2id = list(vocab.keys())
id2word = {i:word for i, word in enumerate(vocab)}

vec = CountVectorizer(analyzer='char', ngram_range=(1,1))
X = vec.fit_transform(vocab)

file1 = os.path.join(root_data_dir, 'sents_with_mistakes.txt')
file2 = os.path.join(root_data_dir, 'correct_sents.txt')
bad = open(file1, encoding='utf8').read().splitlines()
true = open(file2, encoding='utf8').read().splitlines()

In [20]:
def get_closest_match_with_metric(text, lookup,topn=20, metric=textdistance.levenshtein):

    similarities = Counter()
    for word in lookup:
        similarities[word] = metric.normalized_similarity(text, word)
    return similarities.most_common(topn)


def get_closest_match_vec(text, X, vec, topn=20):
    v = vec.transform([text])

    similarities = cosine_distances(v, X)[0]
    topn = similarities.argsort()[:topn]

    return [(id2word[top], similarities[top]) for top in topn]

def get_closest_hybrid_match(text, X, vec, topn=5, metric=textdistance.damerau_levenshtein):
    candidates = get_closest_match_vec(text, X, vec, topn*4)
    lookup = [cand[0] for cand in candidates]
    closest = get_closest_match_with_metric(text, lookup, topn, metric=metric)

    # Добавим выбор наиболее вероятного слова, если расстояние между словами одинаковое

    distances = []
    for item in closest:
        a = list(item)
        a.append(P(item[0]))
        distances.append(a)

    total = pd.DataFrame(distances, columns = ['word', 'distance', 'probability'])
    total = total.sort_values(by = ['distance', 'probability'], ascending = False).reset_index(drop=True)

    unique_dist = list(total.distance.unique())
    the_most_probable = []
    for i in range(len(total)):
        if total.distance[i] in unique_dist:
            the_most_probable.append(total.word[i])
            the_most_probable.append(total.distance[i])
            unique_dist.remove(total.distance[i])

    return the_most_probable

N = sum(vocab.values())
def P(word, N=N):
    return vocab[word] / N

def predict_mistaken(word, vocab):
    return 0 if word in vocab else 1

In [21]:
# Проверка работы на true и bad
def align_words(sent_1, sent_2):
    tokens_1 = sent_1.lower().split()
    tokens_2 = sent_2.lower().split()

    tokens_1 = [token.strip(punctuation) for token in tokens_1]
    tokens_2 = [token.strip(punctuation) for token in tokens_2]

    tokens_1 = [token for token in tokens_1 if token]
    tokens_2 = [token for token in tokens_2 if token]

    assert len(tokens_1) == len(tokens_2)

    return list(zip(tokens_1, tokens_2))

mistakes = []
total_mistaken = 0
mistaken_fixed = 0

total_correct = 0
correct_broken = 0

total = 0
correct = 0

cashed = {}
for i in range(len(true)):
    word_pairs = align_words(true[i], bad[i])
    for pair in word_pairs:
        if predict_mistaken(pair[1], vocab):
            pred = cashed.get(pair[1], get_closest_hybrid_match(pair[1], X, vec)[0])
            cashed[pair[1]] = pred
        else:
            pred = pair[1]


        if pred == pair[0]:
            correct += 1
        else:
            mistakes.append((pair[0], pair[1], pred))
        total += 1

        if pair[0] == pair[1]:
            total_correct += 1
            if pair[0] != pred:
                correct_broken += 1
        else:
            total_mistaken += 1
            if pair[0] == pred:
                mistaken_fixed += 1

In [24]:
print(get_closest_hybrid_match('карова', X, vec))

['крова', 0.8333333333333334, 'макарова', 0.75, 'кракова', 0.7142857142857143, 'акров', 0.6666666666666667]


In [25]:
print(f"Процент правильных слов: {round((correct/total)*100, 2)} %")
print(f"Процент исправленных ошибок: {round((mistaken_fixed/total_mistaken)*100, 2)} %")
print(f"Процент ошибочно исправленных правильных слов: {round((correct_broken/total_correct)*100, 2)} %")

Процент правильных слов: 84.5 %
Процент исправленных ошибок: 44.33 %
Процент ошибочно исправленных правильных слов: 9.56 %


# **Вывод**

По сравнению с семинаром улучшилось количество исправленных ошибок (на 2.33%). Это не очень много, но всё равно влияние ранжирования вероятностей на результат положительное.

## 2.  Symspell (5 баллов)

Реализуйте алгоритм Symspell. Он похож на алгоритм Норвига, но проще и быстрее. Там к словам применяется только одна операция - удаление символа. Описание алгоритма по шагам:

1) Составляется словарь правильных слов
2) На основе словаря правильных слов составляется словарь удалений - для каждого правильного слова создаются все варианты удалений и создается словарь, где ключ - слово с удалением, а значение - правильное слово
3) Для выбора исправления для слова с опечаткой генерируются все варианты удаления, из них выбираются те, что есть в словаре удалений, построенного на шаге 2. Слово с опечаткой заменяется на правильное слово, соответствующее варианту удаления
4) Если в словаре удалений есть несколько вариантов, то выбирается удаление, которому соответствует наиболее вероятное правильное слово


Оцените качество полученного алгоритма теми же тремя метриками.

In [26]:
from collections import Counter

In [40]:
vocab = Counter(re.findall('[а-яА-ЯёЁ]+', corpus.lower()))
vocab_top = {word:count for word, count in vocab.items() if count > 5}


In [54]:
def create_dictionary(words):
    return Counter(words)

# создание словаря удалений
def create_deletion_dictionary(words):
    deletion_dict = {}
    for word in words:
        for i in range(len(word)):
            deletion = word[:i] + word[i+1:]
            if deletion not in deletion_dict:
                deletion_dict[deletion] = [word]
            else:
                deletion_dict[deletion].append(word)
    return deletion_dict

# исправление слова с опечаткой
def correct_word(word, dictionary, deletion_dict):
    if word in dictionary:
        return word

    suggestions = []
    for i in range(len(word)):
        deletion = word[:i] + word[i+1:]
        if deletion in deletion_dict:
            suggestions += deletion_dict[deletion]

    if len(suggestions) == 0:
        return word

    # выбор наиболее вероятного исправления
    suggestion_count = Counter(suggestions)
    suggestion = suggestion_count.most_common(1)[0][0]

    return suggestion


In [55]:
# пример использования
dictionary = create_dictionary(vocab)
deletion_dict = create_deletion_dictionary(vocab)
print(correct_word("сабака", dictionary, deletion_dict))
print(correct_word("ваще", dictionary, deletion_dict))
print(correct_word("сонце", dictionary, deletion_dict))

абакан
чаще
конце


## Оценка

In [50]:
file1 = os.path.join(root_data_dir, 'sents_with_mistakes.txt')
file2 = os.path.join(root_data_dir, 'correct_sents.txt')
bad = open(file1, encoding='utf8').read().splitlines()
true = open(file2, encoding='utf8').read().splitlines()

def align_words(sent_1, sent_2):
    tokens_1 = sent_1.lower().split()
    tokens_2 = sent_2.lower().split()

    tokens_1 = [token.strip(punctuation) for token in tokens_1]
    tokens_2 = [token.strip(punctuation) for token in tokens_2]

    tokens_1 = [token for token in tokens_1 if token]
    tokens_2 = [token for token in tokens_2 if token]

    assert len(tokens_1) == len(tokens_2)

    return list(zip(tokens_1, tokens_2))

correct = 0
total = 0

total_mistaken = 0
mistaken_fixed = 0

total_correct = 0
correct_broken = 0

cashed = {}
for i in tqdm(range(len(true))):
    word_pairs = align_words(true[i], bad[i])
    for pair in word_pairs:

        predicted = cashed.get(pair[1], correct_word(pair[1],  dictionary, deletion_dict))
        cashed[pair[1]] = predicted


        if predicted == pair[0]:
            correct += 1
        total += 1

        if pair[0] == pair[1]:
            total_correct += 1
            if pair[0] !=  predicted:
                correct_broken += 1
        else:
            total_mistaken += 1
            if pair[0] == predicted:
                mistaken_fixed += 1

print(f"Процент правильных слов: {round((correct/total)*100, 2)} %")
print(f"Процент исправленных ошибок: {round((mistaken_fixed/total_mistaken)*100, 2)} %")
print(f"Процент ошибочно исправленных правильных слов: {round((correct_broken/total_correct)*100, 2)} %")

  0%|          | 0/915 [00:00<?, ?it/s]

Процент правильных слов: 85.11 %
Процент исправленных ошибок: 16.69 %
Процент ошибочно исправленных правильных слов: 4.77 %


# **Вывод**

Symspell работает значительно хуже, чем алгоритм Норвига. Скорее всего, это вызвано тем, что мы работаем только с удалением, хотя исправления также бывают связаны с заменой, вставкой или перестановкой.

## *3. Чтение. (2 балла)

Прочитайте эту главу в книге Speech and Language Processing - https://web.stanford.edu/~jurafsky/slp3/2.pdf .
Ответьте на следующий вопрос:

1. Что такое Byte-Pair Encoding (опишите по-русски, как минимум 10 предложениями)?

*Это задание не связано напрямую с исправлением опечаток, но это важная тема, к которой мы вернемся в конце курса


   Кодирование пар байтов - это алгоритм токенизации текста, который отличается от более простых методов разбиения на слова по пробелам. Преимущество этого алогоритма в том, что токены не всегда соответствуют словам. Это полезно при работе с неологизмами или несуществующими словами.

   Алгоритм состоит из двух шагов. На первом создается словарь, включающий все возможные сочетания символов. Это происходит в ходе многократных прогонов по корпусу: самые частотные рядом стоящие символы заменяются на один общий - и так добавляются в промежуточный корпус. Следующий прогон осуществляется по учебному корпусу - и так далее, пока не образуется окончательный словарь со всеми возможными сочетаниями. В сочетания входят как слова так и части слов (например, приставки).

   На втором шаге используется парсер токенов, использующий этот словарь для разбиения текста на символы. В результате большая часть слов из тестовых данных будет представлена как полноценные символы, но редкие слова могут быть представлены как комбинации символов.

